<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-for-Professionals/blob/main/EY2024/C6_Function_calling_with_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrating tools/function with LLM using Azure OpenAI

In [1]:
!pip install openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [2]:
api_key = "62d77bb382974118aec75d84d274bb72"
api_version = "2024-02-01" # "2023-05-15"
azure_endpoint = "https://eygpt24.openai.azure.com/"
deployment_name = "gpt-4o"

In [3]:
import os
from openai import AzureOpenAI
import json
import requests

client = AzureOpenAI(
  azure_endpoint = azure_endpoint,
  api_key=api_key,
  api_version=api_version
)


In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call one or many functions. The Chat Completions API does not call the function; instead, the model generates JSON that you can use to call the function in your code.

## Supported models
Not all model versions are trained with function calling data. Function calling is supported with the following models: gpt-4, gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-4-0613, gpt-3.5-turbo, gpt-3.5-turbo-0125, gpt-3.5-turbo-1106, and gpt-3.5-turbo-0613

In addition, parallel function calls is supported on the following models: gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-3.5-turbo-0125, and gpt-3.5-turbo-1106

In [4]:
# Define the OpenWeatherMap API key
OWM_API_KEY = "29af1cea50a401d8e624eea4660b3f59"

def get_current_weather(location, unit="kelvin"):
    """
    Fetches the current weather information for a given location.

    Parameters:
    - location: str, the name of the location (e.g., "Paris").
    - unit: str, the unit of temperature (default is "kelvin").

    Returns:
    - str: JSON formatted string containing weather information.
    """
    # Construct the API request URL
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={OWM_API_KEY}"

    # Send the API request
    response = requests.get(url)

    # Parse the temperature and weather forecast from the response
    temp = response.json()['main']['temp']
    forecast = [response.json()['weather'][0]['main'], response.json()['weather'][0]['description']]

    # Create a dictionary with the weather information
    weather_info = {
        "location": location,
        "temperature": temp,
        "unit": 'Kelvin',
        "forecast": forecast
    }

    # Return the weather information as a JSON string
    return json.dumps(weather_info)

In [5]:
get_current_weather("Paris")

'{"location": "Paris", "temperature": 287.39, "unit": "Kelvin", "forecast": ["Clouds", "few clouds"]}'

In [6]:
get_current_weather("Manila")

'{"location": "Manila", "temperature": 302.99, "unit": "Kelvin", "forecast": ["Clouds", "broken clouds"]}'

In [8]:
messages = [{"role":"user",'content':"what is AI?"}]
results = client.chat.completions.create(model = deployment_name, messages = messages)
print(results.model_dump_json(indent=2))

{
  "id": "chatcmpl-A8HjjzbU63od9vtjgKBGgtu8uNbmT",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "AI, or Artificial Intelligence, refers to the simulation of human intelligence in machines that are designed to think and learn like humans. These systems can perform tasks such as reasoning, problem-solving, understanding natural language, recognizing patterns, and making decisions. There are several types of AI, including:\n\n1. **Narrow AI or Weak AI**: These are systems designed and trained for a specific task like voice assistants (e.g., Siri, Alexa), recommendation systems (e.g., Netflix, Amazon), or playing games like chess or Go.\n\n2. **General AI or Strong AI**: This type of AI, often depicted in science fiction, would have the ability to understand, learn, and apply intelligence in a generalized way, similar to human capabilities. This remains largely theoretical and has not yet been realized.

In [9]:

# Define tools/functions for the LLM to use
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit","kelvin"]},
                    },
                    "required": ["location"],
                },
            },
        },
    ]

In [11]:
def get_response(messages, tools, model=deployment_name):
    """
    Handles interaction with the LLM, including making function calls if needed.

    Parameters:
    - messages: list, the conversation history.
    - tools: list, the functions/tools available to the LLM.
    - model: str, the model name to use (default is model_name).

    Returns:
    - str: The response from the LLM or function.
    """
    # Create a chat completion with the given messages and tools
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice='auto',
        temperature=0.2,
    )
    #print(response.model_dump_json(indent=2))
    # Get the generated response and tool calls (if any)
    response = response.choices[0].message
    tool_calls = response.tool_calls

    try:
        if tool_calls:
            print("Making a function call")
            # Available functions
            available_functions = {
                "get_current_weather": get_current_weather,
            }
            print(tool_calls)

            # Extend conversation with assistant's reply
            messages.append(response)

            # Handle each function call
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )

            # Get a new response from the model with the function response
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
            )
            return second_response
        else:
            return response.content
    except Exception as e:
        print("Error occurred", e)
        return response

In [12]:
# Example usage of get_response function
messages = [{"role": "user", "content": "Provide a 2 line explanation for AI"}]
response = get_response(messages, tools)
print(response)

Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. It encompasses various technologies, including machine learning, natural language processing, and robotics, to perform tasks that typically require human intelligence.


In [13]:
messages = [{"role": "user", "content": "How is the weather in Tokyo today?"}]
response = get_response(messages, tools)
print(response.choices[0].message.content)

Making a function call
[ChatCompletionMessageToolCall(id='call_jeXjCCqSbJHetwx7GjKf0KIb', function=Function(arguments='{"location":"Tokyo"}', name='get_current_weather'), type='function')]
The weather in Tokyo today is characterized by a few clouds. The temperature is approximately 303.29 Kelvin (around 30.14°C or 86.25°F).
